In [1]:
from torch_geometric.datasets import Planetoid
import torch

In [2]:
dataset_cora = Planetoid(root='./cora/', name='Cora')
# dataset = Planetoid(root='./citeseer',name='Citeseer')
# dataset = Planetoid(root='./pubmed/',name='Pubmed')
print(dataset_cora)

Cora()


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(dataset_cora.num_node_features, 16)
        self.conv2 = SAGEConv(16, dataset_cora.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.softmax(x, dim=1)

        return x

In [5]:
model = Net()
print(model)

Net(
  (conv1): SAGEConv(1433, 16)
  (conv2): SAGEConv(16, 7)
)


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
data = dataset_cora[0].to(device)
print(data)

cpu
Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])


In [7]:
import torch.optim as optim

In [8]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [9]:
model.train()
for epoch in range(200):
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, pred = torch.max(out[data.train_mask], dim=1)
    correct = (pred == data.y[data.train_mask]).sum().item()
    acc = correct/data.train_mask.sum().item()

    print('Epoch {:03d} train_loss: {:.4f} train_acc: {:.4f}'.format(
        epoch, loss.item(), acc))

Epoch 000 train_loss: 1.9452 train_acc: 0.1571
Epoch 001 train_loss: 1.9229 train_acc: 0.4214
Epoch 002 train_loss: 1.8821 train_acc: 0.5643
Epoch 003 train_loss: 1.8269 train_acc: 0.6071
Epoch 004 train_loss: 1.7725 train_acc: 0.5714
Epoch 005 train_loss: 1.7015 train_acc: 0.6857
Epoch 006 train_loss: 1.6359 train_acc: 0.7786
Epoch 007 train_loss: 1.5815 train_acc: 0.8071
Epoch 008 train_loss: 1.5096 train_acc: 0.8786
Epoch 009 train_loss: 1.4612 train_acc: 0.8929
Epoch 010 train_loss: 1.4121 train_acc: 0.9286
Epoch 011 train_loss: 1.3761 train_acc: 0.9429
Epoch 012 train_loss: 1.3267 train_acc: 0.9643
Epoch 013 train_loss: 1.3006 train_acc: 0.9786
Epoch 014 train_loss: 1.2802 train_acc: 0.9857
Epoch 015 train_loss: 1.2341 train_acc: 0.9857
Epoch 016 train_loss: 1.2457 train_acc: 0.9929
Epoch 017 train_loss: 1.2292 train_acc: 0.9929
Epoch 018 train_loss: 1.2242 train_acc: 0.9714
Epoch 019 train_loss: 1.2120 train_acc: 0.9857
Epoch 020 train_loss: 1.2384 train_acc: 0.9500
Epoch 021 tra

Epoch 176 train_loss: 1.1897 train_acc: 0.9786
Epoch 177 train_loss: 1.1754 train_acc: 1.0000
Epoch 178 train_loss: 1.1814 train_acc: 1.0000
Epoch 179 train_loss: 1.1712 train_acc: 1.0000
Epoch 180 train_loss: 1.1826 train_acc: 0.9929
Epoch 181 train_loss: 1.1781 train_acc: 1.0000
Epoch 182 train_loss: 1.1734 train_acc: 1.0000
Epoch 183 train_loss: 1.1754 train_acc: 1.0000
Epoch 184 train_loss: 1.1812 train_acc: 1.0000
Epoch 185 train_loss: 1.1787 train_acc: 1.0000
Epoch 186 train_loss: 1.1731 train_acc: 1.0000
Epoch 187 train_loss: 1.1751 train_acc: 1.0000
Epoch 188 train_loss: 1.1723 train_acc: 1.0000
Epoch 189 train_loss: 1.1713 train_acc: 1.0000
Epoch 190 train_loss: 1.1868 train_acc: 0.9857
Epoch 191 train_loss: 1.1803 train_acc: 1.0000
Epoch 192 train_loss: 1.1746 train_acc: 1.0000
Epoch 193 train_loss: 1.1735 train_acc: 1.0000
Epoch 194 train_loss: 1.1724 train_acc: 1.0000
Epoch 195 train_loss: 1.1814 train_acc: 0.9929
Epoch 196 train_loss: 1.1749 train_acc: 0.9929
Epoch 197 tra

In [10]:
model.eval()
out = model(data)
loss = criterion(out[data.test_mask], data.y[data.test_mask])
_, pred = torch.max(out[data.test_mask], dim=1)
correct = (pred == data.y[data.test_mask]).sum().item()
acc = correct/data.test_mask.sum().item()
print("test_loss: {:.4f} test_acc: {:.4f}".format(loss.item(), acc))

test_loss: 1.3855 test_acc: 0.8040
